****MODELO PARA RECOMENDACIÓN DE PELÍCULAS****

En este notebook se creará un modelo para recomendación de películas basada en contenido.

El usuario deberá ingresar el título de una película.

El modelo devolverá 5 películas similares.

Basado en las conclusiones del EDA, se construye un modelo basado en contenido, teniendo en cuenta el overview de las películas. 

Para ello se hará uso de un modelo de KNN y se calificarán los ítems de acuerdo a similitud de cosenos.

In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud, STOPWORDS
import numpy as np
#from pandas.io.json import json_normalize
#import ast
#import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors



sns.set()

Para pasarlo a la API, acá debería empezar mi función.

**Paso 1: Preparación de los datos**

In [2]:
movies = pd.read_csv('Movies ETL.csv', sep=',')
movies.head()

,Unnamed: 0,belongs_to_collection,budget,genres,id,original_language,overview,popularity,production_companies,production_countries,...,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,release_year,return
0,0,Toy Story Collection,30000000.0,"['Animation', 'Comedy', 'Family']",862.0,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,['Pixar Animation Studios'],['US'],...,373554033.0,81.0,['en'],Released,NaN,Toy Story,7.7,5415.0,1995,12.451801
1,1,NaN,65000000.0,"['Adventure', 'Fantasy', 'Family']",8844.0,en,When siblings Judy and Peter discover an encha...,17.015539,"['TriStar Pictures', 'Teitler Film', 'Intersco...",['US'],...,262797249.0,104.0,"['en', 'fr']",Released,Roll the dice and unleash the excitement!,Jumanji,6.9,2413.0,1995,4.043035
2,2,Grumpy Old Men Collection,0.0,"['Romance', 'Comedy']",15602.0,en,A family wedding reignites the ancient feud be...,11.712900,"['Warner Bros.', 'Lancaster Gate']",['US'],...,0.0,101.0,['en'],Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,6.5,92.0,1995,0.000000
3,3,NaN,16000000.0,"['Comedy', 'Drama', 'Romance']",31357.0,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,['Twentieth Century Fox Film Corporation'],['US'],...,81452156.0,127.0,['en'],Released,Friends are the people who let you be yourself...,Waiting to Exhale,6.1,34.0,1995,5.090760
4,4,Father of the Bride Collection,0.0,['Comedy'],11862.0,en,Just when George Banks has recovered from his ...,8.387519,"['Sandollar Productions', 'Touchstone Pictures']",['US'],...,76578911.0,106.0,['en'],Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,5.7,173.0,1995,0.000000


In [3]:
# Relleno los valores nulos en 'overview' con una cadena vacía
movies['overview'] = movies['overview'].fillna('')

Voy a reducir el tamaño de la matriz por limitaciones en la capacidad de procesamiento de mi equipo y por limitaciones en Render a la hora de generar la API.

Ordenaré el Dataframe de acuerdo a papularidad, de mayor a menor. Y tomaré lo primeros N registros.

El orden por popularidad lo hago porque, al ser las más conocidas y vistas, es muy probable que hayan escuchado hablar de ellas y, tal vez, las tengas en una lista mental... No viene mal un recordatorio!!!

In [4]:
movies_ordenado = movies.sort_values(by= 'popularity', ascending=False)
N = 5000
#muestra_index = np.random.choice(range(len(movies)), size=muestra, replace=False)
movies_muestra = movies_ordenado[['title','overview', 'popularity']].head(N)
#movies_muestra = movies[['title','overview', 'popularity']].head(N)

#reseteo el index
movies_muestra = movies_muestra.reset_index(drop=True)

In [5]:
movies_muestra.shape

(5000, 3)

In [6]:
movies_muestra.head(20)

,title,overview,popularity
0,Minions,"Minions Stuart, Kevin and Bob are recruited by...",547.488298
1,Wonder Woman,An Amazon princess comes to the world of Man t...,294.337037
2,Beauty and the Beast,A live-action adaptation of Disney's version o...,287.253654
3,Baby Driver,After being coerced into working for a crime b...,228.032744
4,Big Hero 6,The special bond that develops between plus-si...,213.849907
5,Deadpool,Deadpool tells the origin story of former Spec...,187.860492
6,Guardians of the Galaxy Vol. 2,The Guardians must fight to keep their newfoun...,185.330992
7,Avatar,"In the 22nd century, a paraplegic Marine is di...",185.070892
8,John Wick,Ex-lunatic John Wick comes off his meds to tra...,183.870374
9,Gone Girl,With his wife's disappearance having become th...,154.801009


**Paso 2: Vectorización del texto**

In [7]:
tfidf = TfidfVectorizer(stop_words='english')
overview_matrix = tfidf.fit_transform(movies_muestra['overview'])

In [8]:
overview_matrix.shape

(5000, 21054)

**Paso 3: Construcción del modelo 1: KNN**

Voy a usar KNN con K=5

In [9]:
#Instanciar y entrenar el modelo.
k = 5
model = NearestNeighbors(metric='cosine', algorithm='brute')
#model.fit(overview_matrix)
model.fit(overview_matrix)



NearestNeighbors(algorithm='brute', metric='cosine')

**Paso 4: Función de Recomendación**

In [10]:
def recomendacion(titulo):
#Se ingresa el nombre de una película y te recomienda las similares en una lista de 5 valores.

    #Obtener el índice de la película ingresada dentro del DF movies_muestra. Esto es para encontrar la posición correspondiente dentro de la matriz de similitud.
    movie_index = movies_muestra[movies_muestra['title'] == titulo].index[0]

    #Encuentro los vecinos más cercanos gracias al modelo entrenado (model)
    distancias, indices = model.kneighbors(overview_matrix[movie_index], n_neighbors=k+1)

    #Ordenar el listado de vecinos de acuerdo a las distancias.
    similar_movies = sorted(list(zip(indices.squeeze().tolist(), distancias.squeeze().tolist())), key=lambda x: x[1])[1:]

    #Obtener sólo el índice de las películas.
    similar_movie_indices = [i[0] for i in similar_movies]

    #Con el índice anterior, busco los títulos de las películas recomendadas en el DF reducido y lo devuelvo como lista.
    recommended_movies = movies_muestra.iloc[similar_movie_indices]['title'].tolist()
    
    return recommended_movies


In [11]:
recomendacion('The Dark Knight')

['The Dark Knight Rises',
 'Batman Returns',
 'Batman Forever',
 'Batman: The Dark Knight Returns, Part 2',
 'Batman: The Dark Knight Returns, Part 1']

        OTRO ENSAYO

**Paso 3: Contrucción del Modelo 2**

In [12]:
#Primero, defino una forma de identificar el índice dado el título de la película
indice = pd.Series(movies_muestra.index, index=movies_muestra['title']).drop_duplicates()

Defino una función de recomendación que tendrá los siguientes pasos:

1- Obtener el índice de la película dado su título.

2- Obtener un listado de todas las películas con su score de similitud de coseno respecto a la película dada.

3- Ordenar la lista en función del score.

4- Obtener el top 5.

5- Returnar los títulos de las 5 películas

In [13]:
similarity_matrix = cosine_similarity(overview_matrix)

In [14]:
def recomendacion(titulo, cosine_sim=similarity_matrix):
    idx = indice[titulo]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    sim_scores = sim_scores[1:6]

    movie_indices = [i[0] for i in sim_scores]

    return movies_muestra['title'].iloc[movie_indices]


In [15]:
recomendacion('Ace Ventura: When Nature Calls')

4634              The Golden Child
3478             Failure to Launch
2240                    Pathfinder
2141    Ace Ventura: Pet Detective
258                        Top Gun
Name: title, dtype: object

****Los resultados obtenidos son exactamente iguales a los del modelo 1****

**COMENTARIOS**

De haber tenido más tiempo, me hubiese gustado explorar opciones recomendando por directores y actores.